In [1]:
import torch
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [2]:
from audioop import mul
import numpy as np
import torch
from sklearn import preprocessing
import os
from pathlib import Path
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from sklearn.preprocessing import OneHotEncoder

In [3]:
torch.cuda.FloatTensor(10,1).normal_(0,1)

tensor([[-0.4545],
        [-0.7868],
        [ 0.9084],
        [-1.6203],
        [ 0.1068],
        [ 0.2312],
        [ 0.4612],
        [ 1.0168],
        [-0.8705],
        [-0.9826]], device='cuda:0')

In [5]:
# image_feature
vis_emb = np.load(r"C:\HolisticVideoUnderstanding\sampled_test\applying cream\0PgfOKP1Ow0_66\InceptionResnetV2_MaxPooling.npz")["InceptionResnetV2"]
print(vis_emb.shape)

# textual_feature
txt_emb = np.load(r"C:\HolisticVideoUnderstanding\sampled_test\applying cream\0PgfOKP1Ow0_66\Elmo_Mean.npz")["Elmo"]
print(txt_emb.shape)

(1536,)
(1024,)


In [51]:
# Dataset Loader
class HVUDataset(Dataset):
    def __init__(self, n_trial):
        self.total = n_trial
        
    def __len__(self):        
        return self.total # 4
    
    # this function will return one n-way, k-shot set
    def __getitem__(self,idx):
        
        #return torch.randperm(idx)
        return "erol___"+str(idx)

In [52]:
ds = HVUDataset(n_trial=30)
    
train_loader = DataLoader(dataset= ds, batch_size = 10, shuffle=True, num_workers = 0) 

In [62]:
for local_batch in train_loader:
    print(local_batch)    

['erol___24', 'erol___5', 'erol___19', 'erol___2', 'erol___22', 'erol___15', 'erol___29', 'erol___0', 'erol___26', 'erol___14']
['erol___8', 'erol___25', 'erol___7', 'erol___20', 'erol___6', 'erol___16', 'erol___1', 'erol___17', 'erol___12', 'erol___10']
['erol___27', 'erol___11', 'erol___23', 'erol___21', 'erol___3', 'erol___13', 'erol___9', 'erol___4', 'erol___28', 'erol___18']


In [4]:
empty = []
empty_test = []

a = [np.ones(3)*1, np.ones(3)*2, np.ones(3)*3, np.ones(3)*4]
b = [np.ones(3)*10, np.ones(3)*20, np.ones(3)*30, np.ones(3)*40]

empty.extend(a[:-1])
empty_test.extend(a[-1:])

empty.extend(b[:-1])
empty_test.extend(b[-1:])


print(empty)
print(empty_test)

[array([1., 1., 1.]), array([2., 2., 2.]), array([3., 3., 3.]), array([10., 10., 10.]), array([20., 20., 20.]), array([30., 30., 30.])]
[array([4., 4., 4.]), array([40., 40., 40.])]


In [9]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

normalizer = MinMaxScaler()

In [30]:
visual_f = np.load("C:/HolisticVideoUnderstanding/uniform_train/air drumming/qF1dEUpUbNA_80/InceptionResnetV2_MaxPooling.npz")['InceptionResnetV2']
textual_f = np.load("C:/HolisticVideoUnderstanding/uniform_train/air drumming/qF1dEUpUbNA_80/Elmo_Mean.npz")['Elmo']

In [25]:
np.mean(visual_f), np.min(visual_f), np.max(visual_f)

(0.72966594, 0.07658113, 3.2027264)

In [28]:
new_visual_f = normalizer.fit_transform(np.float32(visual_f).reshape(-1, 1))

In [29]:
np.mean(new_visual_f), np.min(new_visual_f), np.max(new_visual_f)

(0.20891058, 0.0, 1.0)

In [31]:
np.mean(textual_f), np.min(textual_f), np.max(textual_f)

(-0.013344703, -2.2783346, 1.8741663)

In [36]:
new_textual_f = normalizer.fit_transform(np.float32(textual_f).reshape(-1, 1))

In [37]:
np.mean(new_textual_f), np.min(new_textual_f), np.max(new_textual_f)

(0.545452, 0.0, 1.0)

#### 1) Distribution Aware Classification

In [3]:
from sklearn.metrics import classification_report

In [4]:
gt = [1,1,1,1,1,2,2,2,2,2,0,0,0,0,0]
n_way = 3
k_shot = 5

# Training Stage
mu_1 = torch.normal(10, 1, (5,64))
mu_2 = torch.normal(-30, 1, (5,64))
mu_3 = torch.normal(170, 1, (5,64))

logvar_1 = torch.normal(10, 1, (5,64))
logvar_2 = torch.normal(-30, 1, (5,64))
logvar_3 = torch.normal(170, 1, (5,64))

mus = torch.cat((mu_1, mu_2, mu_3))
logvars = torch.cat((logvar_1, logvar_2, logvar_3))

# Testing Stage
mu_1 = torch.normal(7, 1, (5,64))
mu_2 = torch.normal(-20, 1, (5,64))
mu_3 = torch.normal(100, 1, (5,64))

logvar_1 = torch.normal(7, 1, (5,64))
logvar_2 = torch.normal(-20, 1, (5,64))
logvar_3 = torch.normal(100, 1, (5,64))

test_mus = torch.cat((mu_1, mu_2, mu_3))
test_logvars = torch.cat((logvar_1, logvar_2, logvar_3))

#mus = torch.rand((15,1,64)).reshape((n_way*k_shot, 64))
#logvars = torch.rand((15,1,64)).reshape((n_way*k_shot, 64))
#test_mus = torch.rand((15,1,64)).reshape((n_way*k_shot, 64))
#test_logvars = torch.rand((15,1,64)).reshape((n_way*k_shot, 64))

In [5]:
mus.shape, logvars.shape, test_mus.shape, test_logvars.shape

(torch.Size([15, 64]),
 torch.Size([15, 64]),
 torch.Size([15, 64]),
 torch.Size([15, 64]))

##### 1.a) Class mus, logvar construction stage

In [6]:
alpha = 1 / k_shot
avg_train_mus = []
avg_train_vars = []

In [7]:
for i in range(0,len(mus) - 1, k_shot):
    
    avg_mu = np.mean(mus[i: i + k_shot].cpu().detach().numpy(), axis=0)
    
    class_mu = mus[i: i + k_shot].cpu().detach().numpy()
    class_variance = logvars[i: i + k_shot].cpu().detach().numpy()
        
    # sinif ici her bir mu variance al
    avg_var = [0.0] * 64
    for j in range(len(class_variance)):
        
        # For the 1st, 2nd, 3rd solutions            
        if k_shot != 1:
            avg_var += (( 1/k_shot * class_variance[j] ) + (1/k_shot * ((class_mu - avg_mu)**2)))
        # If there is only 1 sample, avg_var equals to that sample's var
        else:
            avg_var += class_variance[j]        
            
    # class oriented var
    avg_train_vars.append(avg_var)
    
    # class oriented mu
    avg_train_mus.append(avg_mu)

In [8]:
avg_train_mus[0].shape, avg_train_mus[1].shape, avg_train_mus[2].shape

((64,), (64,), (64,))

In [9]:
avg_train_vars[0].shape, avg_train_vars[1].shape, avg_train_vars[2].shape

((5, 64), (5, 64), (5, 64))

##### 2) Testing stage

In [10]:
len(avg_train_mus)

3

In [17]:
loss = torch.nn.CrossEntropyLoss()

gt_tensor = np.array(gt, dtype=np.int32)
gt_tensor = torch.from_numpy(gt_tensor).type(torch.long).reshape(-1,1)


In [22]:
test_pred_labels = []

for i in range(len(test_mus)):
    test_dist = torch.distributions.Normal(test_mus[i],test_logvars[i])

    local_scores = []
    for j in range(len(avg_train_mus)):
        
        # KL divergence
        train_dist = torch.distributions.Normal(torch.Tensor(avg_train_mus[j]), torch.Tensor(avg_train_vars[j]))
        scr = torch.distributions.kl_divergence(test_dist, train_dist).mean()
        #print("{}.th sample, {}.th class, SCORE:{}".format(i,j,scr))
        local_scores.append(scr)
        
    #print(local_scores)
    
    local_similarities = torch.from_numpy(np.array([-1 * elem for elem in local_scores])).reshape(1,-1)
    
    # categorical cross entropy loss    
    #print(local_similarities)    
    #print(gt_tensor[i])
    cc_loss = loss(local_similarities, gt_tensor[i])
    
    # min of kl divergence for class decision
    pred_cls_idx = np.argmin(local_scores)
    
    print(pred_cls_idx)
    print(cc_loss)
    print("----------")
        
    test_pred_labels.append(pred_cls_idx)

0
tensor(1.8231)
----------
0
tensor(1.8080)
----------
0
tensor(1.8079)
----------
0
tensor(1.7971)
----------
0
tensor(1.8003)
----------
1
tensor(2.2241)
----------
1
tensor(2.2272)
----------
1
tensor(2.2249)
----------
1
tensor(2.2248)
----------
1
tensor(2.2199)
----------
2
tensor(76.8491)
----------
2
tensor(77.0428)
----------
2
tensor(77.3163)
----------
2
tensor(77.2453)
----------
2
tensor(77.2331)
----------


In [15]:
raw = [-0.2025, -1.7995, -3.1739]
raw_exp = [np.exp(elem) for elem in raw]
probabilities = [elem/np.sum(raw_exp) for elem in raw_exp]

probabilities

[0.7976169420317689, 0.16151991692720932, 0.04086314104102191]

In [13]:
print("Pred \t G.T.")
for i in range(len(test_pred_labels)):
    print("{} \t {}".format(test_pred_labels[i], gt[i]))

Pred 	 G.T.
0 	 1
0 	 1
0 	 1
0 	 1
0 	 1
1 	 2
1 	 2
1 	 2
1 	 2
1 	 2
2 	 0
2 	 0
2 	 0
2 	 0
2 	 0


In [14]:
print(classification_report(gt, test_pred_labels))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       5.0
           1       0.00      0.00      0.00       5.0
           2       0.00      0.00      0.00       5.0

    accuracy                           0.00      15.0
   macro avg       0.00      0.00      0.00      15.0
weighted avg       0.00      0.00      0.00      15.0

